In [1]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import torchvision.transforms as transforms

from PIL import Image


In [2]:
BATCH_SIZE = 8

In [6]:
from ResNet import ResNet50

net = ResNet50(10)
# net.eval()


In [15]:
net.load_state_dict(torch.load('resnet50.pth'))
net.to('cuda')
net.eval()

/tmp/ipykernel_95997/1309475102.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load('resnet50.pth'))


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (batch_norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (max_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (batch_norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (batch_norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
      (batch_norm3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (i_downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm2d(256, eps=1e-05, m

In [17]:
class Dataset():
    def __init__(self, list_image_path, list_txt, train: bool):
        # Initialize image paths and corresponding texts
        self.image_path = list_image_path
        # Tokenize text using CLIP's tokenizer
        self.label  = list_txt

        self.train = train

        self.transform_train = transforms.Compose([
            # transforms.RandomHorizontalFlip(),
            # transforms.RandomCrop(32, padding=4),
            transforms.Resize(size=[256,256]),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ])

        self.transform_test = transforms.Compose([
            transforms.Resize(size=[256,256]),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ])

    def __len__(self):
        return len(self.label)

    def __getitem__(self, idx):
        # Preprocess image using CLIP's preprocessing function
        if self.train:
            image = self.transform_train(Image.open(self.image_path[idx]))
        else:
            image = self.transform_test(Image.open(self.image_path[idx]))

        label = self.label[idx]
        return image, label

In [18]:
df_test = pd.read_csv('data_balanced/test/data_test.csv')

str_labels = list(set(df_test['label']))

img_path = df_test['image_path'].tolist()
labels = df_test['int_label'].tolist()

print(f'length of test split: {len(labels)}')

dataset = Dataset(img_path, labels, train=False)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

print(f'Num of Batches in test dataloader: {len(dataloader)}')

length of test split: 137
Num of Batches in test dataloader: 18


In [20]:
y_true = []
y_pred = []

correct = 0
total = 0
with torch.no_grad():
    for i, data in enumerate(dataloader):
        images, labels = data
        images, labels = images.to('cuda'), labels.to('cuda')
        outputs = net(images)
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        y_true += [np.array(i.cpu()) for i in labels]
        y_pred += [np.array(i.cpu()) for i in predicted]
        
dataloader_size = (len(dataloader))
print(f'Accuracy on {dataloader_size*BATCH_SIZE} test images:  {100*(correct/total)}%')

Accuracy on 144 test images:  11.678832116788321%


: 